# Grid-based integration on implicit domains

- Classic quadrature schemes work nicely on 1D domains
 
- Extensions to higher dimensional domains run into issues:

  - Break into lower-dim. subintegrals: Limits on the subintegrals?
  - Subdivision into "nice" subdomains of full dim. requires "meshing" (e.g. marching cubes)

- Domains of positive codimension (i.e. a dim. less than the ambient):

  - volume integral (codim = 0); e.g. volume, moment of intertia
  
  $$\text{ vs.}$$
  - surface integral (codim = 1); e.g.surface area, shell moment of inertia
  
  - line/path integral (codim = 2); e.g. path length, work by moving force

- Polygonization methods are unreliable.

- Monte Carlo methods become expensive.

## Consider alternative approach: 

- Inputs: defining function $f$ and integrand $g$
- Compute integral of $g$ over region definbed by $f$

Also handle related problem:

- Inputs: sampled values of $f,g$
  - discrete grid
  - NOT continuously defined


Here we go...

## Problem formulation

Compute the value of the integrals:

$$
\begin{aligned}
\text{Volume (codim = 0)}: &\qquad I_0 &= \iiint\limits_\Omega g(\vec{r}) dV \\
\text{Surface (codim = 1)}: &\qquad I_1 &= \iint\limits_{\partial \Omega} g(\vec{r}) dS \\
\text{Path (codim = 2)}: &\qquad I_2 &= \displaystyle \int\limits_{\partial \Omega_1 \cap \partial \Omega_2} g(\vec{r}) ds \\
\text{Point sum (codim = 3)}: &\qquad I_3 &= \displaystyle \sum\limits_{\partial \Omega_1 \cap \partial \Omega_2 \cap \partial \Omega_3} g(\vec{r}) 
\end{aligned}
$$ 

Domain definitions:  

$$
\begin{aligned}
\Omega &= \{\vec{r} : f(\vec{r}) < 0\} \\
\partial \Omega &= \{\vec{r} : f(\vec{r}) = 0\} \\
\partial \Omega_1 \cap \partial \Omega_2 &= \{\vec{r} : f_1(\vec{r}) =
 f_2(\vec{r}) = 0\} \\
 \partial \Omega_1 \cap \partial \Omega_2  \cap \partial \Omega_3 &= \{\vec{r} : f_1(\vec{r}) =
 f_2(\vec{r}) = f_3(\vec{r}) = 0\}
\end{aligned}
$$

Intuitive to start with codim0 volume integrals

- Quickly get to "old school" voxels
- Break up into "sugar cubes"
- Sum the cubes where $f<0$
  - VERY simple BUT...
  - Cannot fully classify boundary voxels
  - May need gradient bound
- Good for "quick and dirty", but for more accurate results...

### Start with surface integrals ($codim$ 1)

- Allows ready application of Divergence Theorem:

$$\iint\limits_{\partial \Omega} \vec{F} \cdot \hat{n} dS = \iiint\limits_\Omega \nabla \cdot \vec{F} dV$$ (0)

Outward unit normal: 
$\hat{n} = \nabla f / \|\nabla f\|$

Insert $\hat{n} \cdot \hat{n}$ (which equals $1$) into integrand of $I_1$ without changing value (and supress $\vec{r}$ to simplify the notation):

$$I_1 = \iint\limits_{\partial \Omega} g(\vec{r}) dS =\iint\limits_{\partial \Omega} g \, dS = \iint\limits_{\partial \Omega} g \hat{n} \cdot \hat{n} \,dS$$ (1)

Now do a few steps of "mathe-magic"... 

- Apply the divergence theorem with $\vec{F} = g \hat{n}$:

$$I_1 = \iint\limits_{\partial \Omega} (g \hat{n}) \cdot \hat{n} dS = \iiint\limits_{\Omega} \nabla \cdot (g \hat{n}) dV$$ (2)

- Introduce ___occupancy/characteristic/indicator/support function___:

$$
\mathbb{1}_\Omega = \chi(f) = \begin{cases}
   1 &\text{if } f \leq 0 \\
   0 &\text{if } f > 0
\end{cases}
$$ (3)

Support function $\mathbb{1}_\Omega = \chi$:

a) Zeroes out anything outside the region $\Omega$ where $f<0$

b) Allows extending integration  to a box $B$ that contains $\Omega$

c) Integral remains unchanged provided that $\mathbb{1}_\Omega =\chi(f)$ is included in the integrand:

$$I_1 = \iiint\limits_{\Omega} \nabla \cdot (g \hat{n}) dV = \iiint\limits_{B} \bold{1}_\Omega \nabla \cdot (g \hat{n}) dV$$ (4)


- Apply the product rule (or integration by parts):

$$\nabla \cdot (\mathbb{1}_\Omega g \hat{n}) = \nabla \bold{1}_\Omega \cdot (g \hat{n}) + \mathbb{1}_\Omega \nabla \cdot (g \hat{n})
$$

Solve for last term and plug into Eq.(4):

$$\mathbb{1}_\Omega \nabla \cdot (g \hat{n}) =  - \nabla  \cdot (g \hat{n}) + \nabla \cdot (\mathbb{1}_\Omega g \hat{n})
$$


$$I_1 = - \iiint\limits_{B} \nabla \mathbb{1}_\Omega \cdot (g \hat{n}) dV + \iiint\limits_{B} \nabla \cdot (\mathbb{1}_\Omega g \hat{n}) dV$$ (5)

Move scalar $g$ outside the dot product in  first term and apply divergence theorem (in opposite direction with $\vec{F} = \mathbb{1}_\Omega g \hat{n}$) to second term:

$$I_1 = - \iiint\limits_{B} g \nabla \bold{1}_\Omega \cdot \hat{n} dV + \iint\limits_{\partial B} \mathbb{1}_\Omega g \hat{n} \cdot \hat{n} dS$$ (6)

Since $\Omega \subset B$ and $\mathbb{1}_\Omega \equiv 0$ outside $\Omega$, $\mathbb{1}_\Omega \equiv 0$ on $\partial B$ and the boundary integral vanishes:

$$
\begin{aligned}
I_1 &= - \iiint\limits_{B} g \nabla \mathbb{1}_\Omega \cdot \hat{n} dV + \cancel{\iint\limits_{\partial B} \mathbb{1}_\Omega g \hat{n} \cdot \hat{n} dS} \\
\\
I_1 &= - \iiint\limits_{B} g \nabla \mathbb{1}_\Omega \cdot \hat{n} dV 
\end{aligned}$$ (7)

Substitute for unit normal in terms of  defining function:

$$I_1 = - \iiint\limits_{B} g \nabla \mathbb{1}_\Omega \cdot \frac{\nabla f}{\| \nabla f\|} dV$$ (8)

To be more concrete, let's write the Cartesian coordinate version of this expression:

$$
\begin{aligned}
I_1 = - \iiint\limits_{B} g 
\frac{
    \frac{\partial \mathbb{1}_\Omega}{\partial x} \frac{\partial f}{\partial x} + \frac{\partial \mathbb{1}_\Omega}{\partial y} \frac{\partial f}{\partial y} + \frac{\partial \mathbb{1}_\Omega}{\partial z} \frac{\partial f}{\partial z}
    }{
        \sqrt{\frac{\partial f}{\partial x}^2 +  \frac{\partial f}{\partial y}^2+  \frac{\partial f}{\partial z}^2}} dV \\
        \\
- \iiint\limits_{B} g 
\frac{\partial_x\mathbb{1}_\Omega \partial_xf + \partial_y\bold{1}_\Omega \partial_yf +\partial_z\mathbb{1}_\Omega \partial_zf}{\sqrt{(\partial_xf)^2+(\partial_yf)^2+(\partial_zf)^2}}
     dV
\end{aligned}
$$ (9)

Discretize:
- Integral $\to$ voxel sum
- Derivative $\to$ finite (central) difference:

$$
\begin{aligned}
2h\Delta_xf_{i,j,k} = f_{i+1,j,k}-f_{i-1,j,k}, \qquad&2h\Delta_x (\mathbb{1}_\Omega)_{i,j,k} = f_{i+1,j,k}-f_{i-1,j,k} \\
2h\Delta_yf_{i,j,k} = f_{i,j+1,k}-f_{i,j-1,k}, \qquad&2h\Delta_y (\mathbb{1}_\Omega)_{i,j,k} = f_{i,j+1,k}-f_{i,j-1,k}\\
2h\Delta_zf_{i,j,k} = f_{i,j,k+1}-f_{i,j,k-1}, \qquad&2h\Delta_z (\mathbb{1}_\Omega)_{i,j,k} = f_{i,j,k+1}-f_{i,j,k-1}
\end{aligned}$$

$$ I_1 = -\frac{h^3}{2h} \sum\limits_{i,j,k}
\big[g \frac{\Delta_x\bold{1}_\Omega \, \Delta_xf+\Delta_y \bold{1}_\Omega \, \Delta_yf+\Delta_z \bold{1}_\Omega \, \Delta_zf}{\sqrt{(\Delta_xf)^2+(\Delta_xf)^2+(\Delta_xf)^2}}
    \big]_{i,j,k}
$$ (10)

Voila! (and/or WTF!)

- Do you buy all of this? 
- Or is it just an object lesson about gullibility?
- Why might you be skeptical?
<br><br><br><br><br><br><br><br><br><br><br><br><br>

- Reasons for skepticism

  - Differentiating discontinuous function $\mathbb{1}_\Omega$
  - Blows up when $\nabla f = 0$
  - So tired of hearing about stuff attributed to Gauss!

Can't do much about the last item, but can address the first two:

- Discontinuous $\mathbb{1}_\Omega$
  - We are used to Fourier-style analysis
  - Lack of smoothness causes convergence issues
  - Instead, think of $\nabla \mathbb{1}_\Omega$ as a "distribution" or generalized $\delta$-function
  - The function may not be well-defined locally, but integrals are well-defined (typical in transform methods)
  - More suitable analysis style: wavelets
    - Scaling in both frequency and amplitude
    - Basis can include non-smooth functions
    - Derivatives available through stencil with ___connection coefficients___
    - Central difference estimate used above coincides with connection coefficients for Daubechies wavelets of genus 1
  - Think of the derivative stencil as arising from connection coefficients instead of finite differences, and a sound basis is available for analysis
  - Convergence proof (as $h \to 0$) provided by Resnikoff \& Wells _"Wavelet analysis: the scalable structure of information,"_ Springer, 2012.

- Deal with $\nabla f = 0$:
  - If denominator is 0, zero out contribution from that voxel
  - For reasonable $f$, singular points are isolated
  - Requires piecewise-smooth boundaries (not fractals!)
  - Voxel volume associated with singular points $\to 0$

## What about the other integrals:

### Volume integral (codim 0): $I_0 = \iiint\limits_{\Omega} g dV$
  - Write integrand as divergence of a vector potential:
  
  $$g = \nabla \cdot \Phi$$
  
  Volume: <br>$g\equiv 1 \implies \Phi = x \hat{e}_x \text{ or } \Phi = y \hat{e}_y\text{ or } \Phi = \frac{1}{2}(x \hat{e}_x+ y \hat{e}_y)$
  <br>$z$-moment of inertia: 
  <br>$g\equiv x^2+y^2 \implies \Phi = \frac{1}{3}(x^3 \hat{e}_x+ y^3 \hat{e}_x)\text{ or } \Phi = z (x^2 + y^2)\, \hat{e}_z$

  - Apply __guess what???__
  
  $$
  \begin{aligned}I_0 &= \iiint\limits_{\Omega} g dV \\
  &= \iiint\limits_{\Omega} \nabla \cdot \Phi \,dV \\
  &= \iiint\limits_{B} \mathbb{1}_\Omega \, \nabla \cdot \Phi \, dV \\
  &= -\iiint\limits_{B} \Phi \cdot  \nabla \mathbb{1}_\Omega \, dV + \iiint\limits_{B} \nabla \cdot (\mathbb{1}_\Omega  \Phi) \, dV \\
  &= -\iiint\limits_{B} \Phi \cdot  \nabla \mathbb{1}_\Omega \, dV + \cancel{\iint\limits_{\partial B} \mathbb{1}_\Omega  \Phi \cdot \hat{n} dS} \\ 
  &= -\iiint\limits_{B} \Phi \cdot  \nabla \mathbb{1}_\Omega \, dV 
  \end{aligned}$$ (11)

Discretization:

$$ I_0 = -\frac{h^3}{2h} \sum\limits_{i,j,k}
\big[\Phi_x \Delta_x\mathbb{1}_\Omega + \Phi_y \Delta_y\mathbb{1}_\Omega +\Phi_z \Delta_z\mathbb{1}_\Omega \big]_{i,j,k}
$$ (12)

### Path integral (codim 2): $I_2 = \displaystyle \int\limits_{\partial \Omega_1 \cap \partial \Omega_2} g(\vec{r}) ds$

This time the naturally associated vector is the unit tangent vector $\hat{t} = \hat{n}_1 \times \hat{n}_2$, and we insert $\hat{t} \cdot \hat{t}$ (again without changing the value of the integral  because $\hat{t} \cdot \hat{t}$ =1) and suppress $\vec{r}$:

$$I_2 = \displaystyle \int\limits_{\partial \Omega_1 \cap \partial \Omega_2} g \, (\hat{t} \cdot \hat{t}) \, ds = \displaystyle \int\limits_{\partial \Omega_1 \cap \partial \Omega_2} (g \, \hat{t}) \cdot \hat{t} \, ds$$

and apply Stokes' Theorem:

$$\displaystyle \int\limits_{\partial S} \vec{u} \cdot \hat{t} \, ds = \displaystyle \int\limits_{S} \nabla \times (\vec{u}) \cdot \hat{n} \, dA$$


with $\vec{u} = g \, \hat{t}$ and $S = \partial \Omega_1 \cap \Omega_2$ (the portion of the surface defined by $f_1$ that lies within the region defined by $f_2$), and $\hat{n} = \hat{n_1}$:

$$I_2 = \displaystyle \iint\limits_{\partial \Omega_1 \cap \Omega_2} \nabla \times (g  \hat{t}) \cdot \hat{n}_1 \, ds$$

This is now in the form of the surface integral $I_1$ (after introducing a clipping function $h$ since the surface of integration is not closed), and we can apply the formulation developed above:

$$I_2 = -\displaystyle \iiint\limits_{B} g \, \nabla \mathbb{1}_{\Omega_1}  \cdot \big(\mathbb{1}_{\Omega_2} \nabla \times (\hat{n}_1 \times \hat{n}_2)\big) \, dV$$







